## --- Day 21: Dirac Dice ---

Play a practice game using the deterministic 100-sided die. The moment either player wins, what do you get if you multiply the score of the losing player by the number of times the die was rolled during the game?

In [1]:
class DiracDice:
    def __init__(self, start_positions, dice_size=100, board_size=10):
        self.DICE = dice_size
        self.BOARD = board_size
        self.positions = start_positions
        self.scores = [0] * len(self.positions)
        self.turn = 0

    def play_round(self, verbose=False):
        self.turn += 1
        rolled_count = 3 * self.turn - 2
        rolls = tuple(self.DICE if (rolled_count+i)%self.DICE==0 else (rolled_count+i)%self.DICE for i in range(3))

        player = 1 if self.turn%2 == 1 else 2
        next_pos = (self.positions[player-1] + sum(rolls)) % 10
        next_pos = self.BOARD if next_pos%self.BOARD == 0 else next_pos
        self.positions[player-1] = next_pos
        self.scores[player-1] += next_pos
        if verbose:
            print(f"Player {player} rolls {'+'.join(str(r) for r in rolls)} and moves to space {next_pos} for a total score of {self.scores[player-1]}")

    def play(self, verbose=False):
        while max(self.scores) < 1000:
            self.play_round(verbose=verbose)

        losing_score = min(self.scores)
        roll_count = 3 * self.turn

        return losing_score * roll_count

In [2]:
ex1 = DiracDice(start_positions=[4,8], dice_size=10)
ex1.play()


739785

In [3]:
# Part 1 solution
p1 = DiracDice(start_positions=[1,3])
p1.play()

897798

## --- Part Two ---

Using your given starting positions, determine every possible outcome. Find the player that wins in more universes; in how many universes does that player win?

__Got unstuck with a useful assist from [/r/adventofcode](https://www.reddit.com/r/adventofcode/comments/rl6p8y/comment/hpinq8a/?utm_source=share&utm_medium=web2x&context=3)!__

In [4]:
from itertools import product
from collections import Counter, defaultdict

class QuantumDirac:
    def __init__(self, start_positions):
        p1_start, p2_start = start_positions
        self.states = {((p1_start, p2_start), (0, 0)): 1}
        self.wins = [0, 0]
        self.moves = Counter([sum(roll) for roll in product((1,2,3), repeat=3)])
        self.turn = 0

    def play_turn(self):
        next_states = defaultdict(int)
        for ((positions, scores), state_count), (move, multiplier) in product(self.states.items(), self.moves.items()):
            next_pos = (positions[self.turn%2] + move) % 10 or 10
            next_score = scores[self.turn%2] + next_pos

            if next_score >= 21:
                self.wins[self.turn%2] += state_count * multiplier
            elif self.turn%2 == 0:
                new_positions = (next_pos, positions[1])
                new_scores = (next_score, scores[1])
                next_states[(new_positions, new_scores)] += state_count * multiplier
            else:
                new_positions = (positions[0], next_pos)
                new_scores = (scores[0], next_score)
                next_states[(new_positions, new_scores)] += state_count * multiplier


        self.turn += 1
        self.states = next_states

    def play(self, verbose=False):
        # Play until all possible outcomes have been reached
        while self.states:
            self.play_turn()
            if verbose:
                print(f"{self.turn} turns, {len(self.states)} states, wins {self.wins}")



In [5]:
# Example
ex2 = QuantumDirac(start_positions=(4, 8))
ex2.play(verbose=True)
assert 444356092776315 == max(ex2.wins)

1 turns, 7 states, wins [0, 0]
2 turns, 49 states, wins [0, 0]
3 turns, 343 states, wins [0, 0]
4 turns, 2401 states, wins [0, 0]
5 turns, 5439 states, wins [3359232, 0]
6 turns, 12210 states, wins [3359232, 26079750]
7 turns, 10120 states, wins [4483386758, 26079750]
8 turns, 8464 states, wins [4483386758, 36354415673]
9 turns, 6440 states, wins [822385675458, 36354415673]
10 turns, 4900 states, wins [822385675458, 5662240503840]
11 turns, 3430 states, wins [35964236460742, 5662240503840]
12 turns, 2401 states, wins [35964236460742, 108506624132684]
13 turns, 1225 states, wins [310533628712037, 108506624132684]
14 turns, 625 states, wins [310533628712037, 323600927041598]
15 turns, 250 states, wins [442449169460193, 323600927041598]
16 turns, 100 states, wins [442449169460193, 341948699931462]
17 turns, 10 states, wins [444356028596613, 341948699931462]
18 turns, 1 states, wins [444356028596613, 341960390180808]
19 turns, 0 states, wins [444356092776315, 341960390180808]


In [6]:
# Part 2 solution
p2 = QuantumDirac(start_positions=[1,3])
p2.play(verbose=True)
print("Solution:", max(p2.wins))

1 turns, 7 states, wins [0, 0]
2 turns, 49 states, wins [0, 0]
3 turns, 343 states, wins [0, 0]
4 turns, 2401 states, wins [0, 0]
5 turns, 4704 states, wins [2877363, 0]
6 turns, 9984 states, wins [2877363, 84990136]
7 turns, 8216 states, wins [3738205925, 84990136]
8 turns, 7268 states, wins [3738205925, 43741733126]
9 turns, 4784 states, wins [442832062105, 43741733126]
10 turns, 3640 states, wins [442832062105, 1842440371926]
11 turns, 2170 states, wins [11660881466544, 1842440371926]
12 turns, 1519 states, wins [11660881466544, 14467838452485]
13 turns, 735 states, wins [42905421787488, 14467838452485]
14 turns, 375 states, wins [42905421787488, 22307603586685]
15 turns, 75 states, wins [48851319610912, 22307603586685]
16 turns, 30 states, wins [48851319610912, 22432440913119]
17 turns, 0 states, wins [48868319769358, 22432440913119]
Solution: 48868319769358
